In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MultiLabelBinarizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras import layers

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Londri/Dataset/Dataset_Laundry (1).csv", error_bad_lines=False)

<ipython-input-4-98e91da9f790>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("/content/drive/MyDrive/Londri/Dataset/Dataset_Laundry (1).csv", error_bad_lines=False)
Skipping line 351: expected 30 fields, saw 49
Skipping line 361: expected 30 fields, saw 38
Skipping line 709: expected 30 fields, saw 53



In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
worksheet = gc.open('Dataset_Laundry').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

df = pd.DataFrame.from_records(rows[1:], columns=rows[0])

In [ ]:
df

,link,title,category,address,open_hours,popular_times,website,phone,plus_code,review_count,...,price_range,data_id,images,reservations,order_online,menu,owner,complete_address,about,user_reviews
0,https://www.google.com/maps/place/Lavanderia/d...,Lavanderia,Laundry service,"Jl. Boulevard No.28 f, Masale, Panakkukang, Ma...","{""Friday"":[""7:30 AM–9 PM""],""Monday"":[""7:30 AM–...","{""Friday"":{""10"":50,""11"":50,""12"":31,""13"":43,""14...",,,"RCVR+Q3 Masale, Makassar City, South Sulawesi",95,...,,0x2dbee2bf4e64e56f:0xf87e6d7309d11f1,"[{""title"":""All"",""image"":""https://lh5.googleuse...",null,null,"{""link"":"""",""source"":""""}","{""id"":""103816956691432943282"",""name"":""Lavander...","{""borough"":""Masale, Panakkukang"",""street"":""Jl....","[{""id"":""service_options"",""name"":""Service optio...","[{""Name"":""Ade Lhinglhung"",""ProfilePicture"":""ht..."
1,https://www.google.com/maps/place/LaundrySaja/...,LaundrySaja,Laundry service,"Jl. Kakatua No.30, Mariso, Makassar City, Sout...","{""Friday"":[""7 AM–9 PM""],""Monday"":[""7 AM–9 PM""]...","{""Friday"":{""10"":15,""11"":0,""12"":0,""13"":0,""14"":0...",https://www.instagram.com/laundrysaja_id/,0853-9755-6884,"RCQ7+H6 Mariso, Makassar City, South Sulawesi",9,...,,0x2dbf1dd8c0928c55:0x458ab705d5e9ef46,"[{""title"":""All"",""image"":""https://lh5.googleuse...",null,null,"{""link"":"""",""source"":""""}","{""id"":""110461900888172896954"",""name"":""LaundryS...","{""borough"":""Mariso"",""street"":""Jl. Kakatua No.3...","[{""id"":""service_options"",""name"":""Service optio...","[{""Name"":""Refly Nakcoc"",""ProfilePicture"":""http..."
2,https://www.google.com/maps/place/Zap+Laundry/...,Zap Laundry,Laundry service,"Di Dalam SPBU Pintu 1 Unhas, Jl. Perintis Keme...","{""Friday"":[""7 AM–9 PM""],""Monday"":[""7 AM–9 PM""]...","{""Friday"":{""10"":75,""11"":61,""12"":42,""13"":37,""14...",http://www.zaplaundry.com/,0878-6999-6679,"VF5Q+7J Tamalanrea Jaya, Makassar City, South ...",140,...,,0x2dbee34a5d0d5a6b:0x876d2bed78883721,"[{""title"":""All"",""image"":""https://lh5.googleuse...",null,null,"{""link"":"""",""source"":""""}","{""id"":""104773887339071515463"",""name"":""Zap Laun...","{""borough"":""Tamalanrea Jaya, Tamalanrea"",""stre...","[{""id"":""accessibility"",""name"":""Accessibility"",...","[{""Name"":""Triwiyanti Triwiyanti"",""ProfilePictu..."
3,https://www.google.com/maps/place/QnC+Laundry+...,QnC Laundry Toddopuli,Laundry service,"Jl. Toddopuli Raya No.8, Kassi-Kassi, Rappocin...","{""Friday"":[""7 AM–12 PM"",""1–10 PM""],""Monday"":[""...","{""Friday"":{""10"":33,""11"":43,""12"":0,""13"":31,""14""...",https://www.qnclaundry.net/,0882-0224-14173,"RCMX+8J Kassi-Kassi, Makassar City, South Sula...",598,...,,0x2dbee39f4bb2b275:0xaad8234ed3dd702b,"[{""title"":""All"",""image"":""https://lh5.googleuse...",null,"[{""link"":""https://api.whatsapp.com/send/?phone...","{""link"":"""",""source"":""""}","{""id"":""110421932829577435298"",""name"":""QnC Laun...","{""borough"":""Kassi-Kassi, Rappocini"",""street"":""...","[{""id"":""service_options"",""name"":""Service optio...","[{""Name"":""isra stimiyapmi"",""ProfilePicture"":""h..."
4,https://www.google.com/maps/place/Laundromad+M...,Laundromad Makassar,Laundry service,"Jl. Bumi Tamalanrea Permai No.1A, Tamalanrea, ...","{""Friday"":[""8 AM–11 PM""],""Monday"":[""8 AM–11 PM...","{""Friday"":{""10"":26,""11"":26,""12"":26,""13"":42,""14...",,0852-5601-0299,"VF9X+7X Tamalanrea, Makassar City, South Sulawesi",75,...,,0x2dbefcae31bb3e63:0xa51a7ff56c9d570e,"[{""title"":""All"",""image"":""https://lh5.googleuse...",null,null,"{""link"":"""",""source"":""""}","{""id"":""103237755937557818310"",""name"":""Laundrom...","{""borough"":""Tamalanrea"",""street"":""Jl. Bumi Tam...","[{""id"":""accessibility"",""name"":""Accessibility"",...","[{""Name"":""andi adinda"",""ProfilePicture"":""https..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,https://www.google.com/maps/place/

In [ ]:
df = df.drop(columns=['link', 'open_hours', 'popular_times', 'address', 'website', 'phone', 'plus_code', 'cid', 'status', 'complete_address', 'descriptions', 'reviews_link', 'thumbnail', 'timezone', 'price_range', 'review_count', 'reviews_per_rating', 'data_id', 'images', 'reservations', 'order_online', 'menu', 'owner', 'about', 'user_reviews'])
df

,title,category,review_rating,latitude,longitude
0,Lavanderia,Laundry service,4.9,-5.155502,119.440242
1,LaundrySaja,Laundry service,4.6,-5.161116,119.413019
2,Zap Laundry,Laundry service,4.3,-5.141826,119.489119
3,QnC Laundry Toddopuli,Laundry service,4.9,-5.166727,119.449074
4,Laundromad Makassar,Laundry service,4.5,-5.131841,119.499985
...,...,...,...,...,...
808,Laundry,Laundry,0,3.5773,98.703334
809,Laundry dry & clean,Laundry,0,3.528993,98.637205
810,Michelle Laundry & Dry Clean,Laundry,5,3.615383,98.697644
811,Laundry,Janitorial service,5,3.593911,98.712397


In [ ]:
df['category'] = df.apply(lambda row: random.sample(['Setrika saja', 'Dryclean', 'Cuci saja', 'Komplit', 'Lainnya'],
                                                   max(1, random.randint(1, 5))), axis=1)
df

,title,category,review_rating,latitude,longitude
0,Lavanderia,"[Setrika saja, Dryclean]",4.9,-5.155502,119.440242
1,LaundrySaja,[Dryclean],4.6,-5.161116,119.413019
2,Zap Laundry,"[Lainnya, Komplit, Cuci saja, Setrika saja, Dr...",4.3,-5.141826,119.489119
3,QnC Laundry Toddopuli,"[Komplit, Setrika saja, Cuci saja, Dryclean]",4.9,-5.166727,119.449074
4,Laundromad Makassar,"[Komplit, Dryclean, Lainnya, Cuci saja, Setrik...",4.5,-5.131841,119.499985
...,...,...,...,...,...
808,Laundry,"[Cuci saja, Komplit, Lainnya, Setrika saja]",0,3.5773,98.703334
809,Laundry dry & clean,"[Cuci saja, Komplit, Lainnya, Dryclean]",0,3.528993,98.637205
810,Michelle Laundry & Dry Clean,[Setrika saja],5,3.615383,98.697644
811,Laundry,"[Komplit, Lainnya, Setrika saja, Dryclean]",5,3.593911,98.712397


In [ ]:
df.to_csv('/content/drive/MyDrive/Londri/Dataset/laundry_dataset.csv', index=False)

In [ ]:
def process_address(address):
    # Extracting borough and splitting on comma
    borough_parts = address['borough'].split(', ')

    # Keeping only the part after the comma
    address['borough'] = borough_parts[-1]

    # Removing "City" from the city name
    address['city'] = address['city'].replace(' City', '')

    # Selecting only required keys
    return pd.Series({'borough': address['borough'], 'city': address['city'], 'state': address['state']})


# Applying the function to the 'complete_address' column and creating new columns
df[['kecamatan', 'kota', 'provinsi']] = df['complete_address'].apply(
    eval).apply(process_address)

# Drop the original 'complete_address' column if needed
df = df.drop('complete_address', axis=1)

# Save the modified DataFrame to a new CSV file
df.to_csv('splitted_new_laundry_dataset.csv', index=False)

KeyError: ignored

In [ ]:
df

,title,category,review_rating,latitude,longitude
0,Lavanderia,"[Setrika saja, Dryclean]",4.9,-5.155502,119.440242
1,LaundrySaja,[Dryclean],4.6,-5.161116,119.413019
2,Zap Laundry,"[Lainnya, Komplit, Cuci saja, Setrika saja, Dr...",4.3,-5.141826,119.489119
3,QnC Laundry Toddopuli,"[Komplit, Setrika saja, Cuci saja, Dryclean]",4.9,-5.166727,119.449074
4,Laundromad Makassar,"[Komplit, Dryclean, Lainnya, Cuci saja, Setrik...",4.5,-5.131841,119.499985
...,...,...,...,...,...
808,Laundry,"[Cuci saja, Komplit, Lainnya, Setrika saja]",0,3.5773,98.703334
809,Laundry dry & clean,"[Cuci saja, Komplit, Lainnya, Dryclean]",0,3.528993,98.637205
810,Michelle Laundry & Dry Clean,[Setrika saja],5,3.615383,98.697644
811,Laundry,"[Komplit, Lainnya, Setrika saja, Dryclean]",5,3.593911,98.712397


In [ ]:
df.to_csv('/content/drive/MyDrive/Londri/Dataset/laundry_dataset.csv', index=False)

In [ ]:
df[['latitude', 'longitude']] = df[['latitude', 'longitude']].astype(float)

In [ ]:
mlb = MultiLabelBinarizer()
encoded_categories = pd.DataFrame(mlb.fit_transform(df['category']), columns=mlb.classes_, index=df.index)

# Concatenate the one-hot encoded categories with the original DataFrame
df_encoded = pd.concat([df, encoded_categories], axis=1)

# Drop the original 'categories' column
df_encoded = df_encoded.drop('category', axis=1)


In [ ]:
df_encoded

,title,review_rating,latitude,longitude,Cuci saja,Dryclean,Komplit,Lainnya,Setrika saja
0,Lavanderia,4.9,-5.155502,119.440242,0,1,0,0,1
1,LaundrySaja,4.6,-5.161116,119.413019,0,1,0,0,0
2,Zap Laundry,4.3,-5.141826,119.489119,1,1,1,1,1
3,QnC Laundry Toddopuli,4.9,-5.166727,119.449074,1,1,1,0,1
4,Laundromad Makassar,4.5,-5.131841,119.499985,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
808,Laundry,0,3.5773,98.703334,1,0,1,1,1
809,Laundry dry & clean,0,3.528993,98.637205,1,1,1,1,0
810,Michelle Laundry & Dry Clean,5,3.615383,98.697644,0,0,0,0,1
811,Laundry,5,3.593911,98.712397,0,1,1,1,1


In [ ]:
df_encoded.to_csv('multibinerizer_dataset.csv', index=False)

In [ ]:
df_final.to_csv('dataset_final.csv')

In [ ]:
df_final

,title,review_rating,latitude,longitude,Cuci saja,Dryclean,Komplit,Lainnya,Setrika saja
0,Lavanderia,4.9,-5.155502,119.440242,1,1,1,1,1
1,LaundrySaja,4.6,-5.161116,119.413019,0,1,0,0,0
2,Zap Laundry,4.3,-5.141826,119.489119,1,1,1,1,1
3,QnC Laundry Toddopuli,4.9,-5.166727,119.449074,1,1,0,1,1
4,Laundromad Makassar,4.5,-5.131841,119.499985,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
808,Laundry,0.0,3.577300,98.703334,1,1,1,1,1
809,Laundry dry & clean,0.0,3.528993,98.637205,0,1,0,0,1
810,Michelle Laundry & Dry Clean,5.0,3.615383,98.697644,1,0,0,1,1
811,Laundry,5.0,3.593911,98.712397,1,0,0,1,0
